In [44]:
!pip install  mlflow 

import sqlite3
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [45]:
DB_PATH = "D:/BITS_SEM2/DMML_Assignment/Task6_FeatureStore/feature_store.db"   
TABLE_NAME = "feature_store"
TARGET_COL = "Churn"

In [46]:
conn = sqlite3.connect(DB_PATH)

df = pd.read_sql(f"SELECT * FROM {TABLE_NAME}", conn)
conn.close()

print(f"Loaded {df.shape[0]} rows and {df.shape[1]} columns")
print("Churn class distribution:")
display(df[TARGET_COL].value_counts())

Loaded 5 rows and 10 columns
Churn class distribution:


Churn
1    3
0    2
Name: count, dtype: int64

In [ ]:
import joblib
drop_cols = ["Churn","customerID"]
df = df.drop(columns=[c for c in drop_cols if c in df.columns])
print("Columns in X:", df.columns.tolist())

# Separate features and target
y = df[TARGET_COL]
X = df.drop(TARGET_COL, axis=1)

# One-hot encode categorical features (if any)
X = pd.get_dummies(X, drop_first=True)

# Sanity checks
print("Columns in X:", X.columns.tolist())
print("Target unique values:", y.unique())
print("Class distribution:\n", y.value_counts())
print("Any duplicate rows?:", df.duplicated().sum())

# Check if target has at least 2 classes
if y.nunique() < 2:
    raise ValueError(f"Target column '{TARGET_COL}' must have at least 2 classes. Got: {y.unique()}")

# Split into train/test sets (stratified)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

models = {
    "LogisticRegression": LogisticRegression(max_iter=1000),
    "RandomForest": RandomForestClassifier(n_estimators=100, random_state=42),
}

# Train, evaluate, and save models
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)
    
    print(f"{name} --> Acc:{acc:.3f} | Prec:{prec:.3f} | Recall:{rec:.3f} | F1:{f1:.3f}")
    
    # Save model and scaler
    joblib.dump(model, f"{name}.pkl")
    joblib.dump(scaler, f"{name}_scaler.pkl")

KeyError: "['Churn', 'customerID'] not found in axis"